# 🔍 Customer Churn Prediction - Proyecto ML

**Paso 1: Entendiendo el problema**

El abandono de clientes (customer churn) es un gran problema para las empresas que ofrecen servicios por suscripción. Este proyecto busca predecir si un cliente se va a dar de baja o no, utilizando datos históricos de sus características y comportamiento.

---

- **Tipo de problema**: Clasificación binaria
- **Dataset**: Público, obtenido de Kaggle  
  URL: https://www.kaggle.com/datasets/muhammadshahidazeem/customer-churn-dataset
- **Objetivo**: Construir un modelo que prediga la variable `churn` (abandono del cliente)



In [1]:
import pandas as pd


In [11]:
# Ruta a los archivos
train_path = "../data_sample/customer_churn_dataset-training-master.csv"
test_path = "../data_sample/customer_churn_dataset-testing-master.csv"

# Cargar datasets
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

# Mostrar forma y primeras filas
print(df_train.shape)
print(df_test.shape)
df_train.head()


(440833, 12)
(64374, 12)


,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.0,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.0,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.0,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.0,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.0,20.0,1.0
